In [4]:
import requests
from bs4 import BeautifulSoup

FBS_URL = "https://en.wikipedia.org/wiki/List_of_NCAA_Division_I_FBS_football_programs"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; NCAA-Ranking-Script/1.0; +https://example.com/)",
    "Accept-Language": "en-US,en;q=0.9",
}


def fetch_fbs_teams():
    """
    Scrape the FBS teams table from Wikipedia and return a list of
    dicts with team_name, mascot (nickname), and conference.
    """
    resp = requests.get(FBS_URL, headers=HEADERS, timeout=30)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    # First wikitable on the page is the FBS program list:
    # School | Nickname | City | State | Enrollment | Current conference | ...
    table = soup.find("table", class_="wikitable")
    if table is None:
        raise RuntimeError("Could not find FBS teams table on the page.")

    teams = []
    for row in table.find_all("tr")[1:]:  # skip header row
        cols = row.find_all("td")
        # We need at least up to the "Current conference" column (index 5)
        if len(cols) < 6:
            continue

        school = cols[0].get_text(strip=True)
        nickname = cols[1].get_text(strip=True)
        conference = cols[5].get_text(strip=True)

        teams.append({
            "team_name": school,
            "mascot": nickname,
            "conference": conference,
        })

    return teams


def build_sample_teams(teams):
    """
    Take list of {team_name, mascot, conference} and build SAMPLE_TEAMS with:
      - BYU as team_id 1
      - All other teams sorted alphabetically and assigned team_id 2,3,...
    """
    # Wikipedia uses 'BYU' as the school name
    byu_entries = [t for t in teams if t["team_name"] == "BYU"]
    if not byu_entries:
        raise RuntimeError("Could not find BYU in FBS list.")
    byu = byu_entries[0]

    # Other teams (exclude BYU)
    others = [t for t in teams if t is not byu]

    # Sort others alphabetically by team_name
    others_sorted = sorted(others, key=lambda t: t["team_name"])

    # Build final ordered list: BYU first, then everyone else
    ordered = [byu] + others_sorted

    # Assign team_id, forcing BYU to 1 by position
    sample_teams = []
    for idx, team in enumerate(ordered, start=1):
        sample_teams.append({
            "team_id": idx,
            "team_name": team["team_name"],
            "mascot": team["mascot"],
            "conference": team["conference"],
        })

    return sample_teams


def format_as_python_list(sample_teams):
    """
    Format as a Python constant block:

    SAMPLE_TEAMS = [
        {"team_id": 1, "team_name": "BYU", "mascot": "Cougars", "conference": "Big 12"},
        ...
    ]
    """
    lines = ["SAMPLE_TEAMS = ["]
    for t in sample_teams:
        line = (
            f'    {{"team_id": {t["team_id"]}, '
            f'"team_name": "{t["team_name"]}", '
            f'"mascot": "{t["mascot"]}", '
            f'"conference": "{t["conference"]}"}},'
        )
        lines.append(line)
    lines.append("]")
    return "\n".join(lines)


def main():
    teams = fetch_fbs_teams()
    sample_teams = build_sample_teams(teams)
    python_block = format_as_python_list(sample_teams)
    print(python_block)


if __name__ == "__main__":
    main()


SAMPLE_TEAMS = [
    {"team_id": 1, "team_name": "BYU", "mascot": "Cougars", "conference": "Big 12"},
    {"team_id": 2, "team_name": "Air Force", "mascot": "Falcons", "conference": "Mountain West"},
    {"team_id": 3, "team_name": "Akron", "mascot": "Zips", "conference": "MAC"},
    {"team_id": 4, "team_name": "Alabama", "mascot": "Crimson Tide", "conference": "SEC"},
    {"team_id": 5, "team_name": "Appalachian State", "mascot": "Mountaineers", "conference": "Sun Belt"},
    {"team_id": 6, "team_name": "Arizona", "mascot": "Wildcats", "conference": "Big 12"},
    {"team_id": 7, "team_name": "Arizona State", "mascot": "Sun Devils", "conference": "Big 12"},
    {"team_id": 8, "team_name": "Arkansas", "mascot": "Razorbacks", "conference": "SEC"},
    {"team_id": 9, "team_name": "Arkansas State", "mascot": "Red Wolves", "conference": "Sun Belt"},
    {"team_id": 10, "team_name": "Army", "mascot": "Black Knights", "conference": "American[d]"},
    {"team_id": 11, "team_name": "Auburn", "m